## आवश्यक लाइब्रेरी आयात करें  
Phi-4 मॉडल को लोड और उपयोग करने के लिए आवश्यक PyTorch और transformers लाइब्रेरी आयात करें।  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## रैंडम सीड सेट करें  
रैंडम सीड सेट करें ताकि अलग-अलग रन में पुनरुत्पादक परिणाम सुनिश्चित हो सकें।


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning मॉडल और टोकनाइज़र लोड करें  
Microsoft Phi-4-mini-flash-reasoning मॉडल और इसका संबंधित टोकनाइज़र Hugging Face से लोड करें। मॉडल को तेज़ इन्फरेंस के लिए CUDA पर लोड किया जाएगा।  


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## इनपुट संदेश तैयार करें  
मॉडल के लिए चैट टेम्पलेट का उपयोग करके एक बातचीत संदेश बनाएं जिसमें एक द्विघात समीकरण का गणितीय प्रश्न हो।  


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## प्रतिक्रिया उत्पन्न करें  
मॉडल से प्रतिक्रिया उत्पन्न करें, जिसमें आउटपुट में नियंत्रित अनियमितता के लिए तापमान और top_p जैसे पैरामीटर शामिल हों।


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## आउटपुट को टेक्स्ट में डिकोड करें  
जनरेट किए गए टोकन अनुक्रमों को वापस मानव-पढ़ने योग्य टेक्स्ट में बदलें, मूल इनपुट टोकन को छोड़कर ताकि केवल मॉडल की प्रतिक्रिया दिखाई दे।  


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
